Chữ ký số RSA bằng Sage Math (Phó Đức Tài, 27/10/2021)

Thuật toán mã hóa công khai RSA đã học trong phần trước còn được sử dụng để tạo ra chữ ký điện tử.



Ví dụ trên lớp:
Alice: pA=29, qA=71, eA=103, dA=647, nA=2059
Bob: pB=29, qB=37, eB=571, dB=835, nB=1073

Alice cần gửi thông điệp m=331 đã mã hóa & kèm chữ ký s=m^dA = 331^647 (mod 2059)

Bob nhận được và kiểm chứng + giải mã.

In [1]:
pA=29;qA=71;eA=103;pB=29;qB=37;eB=571;

In [2]:
nA=pA*qA;nB=pB*qB

In [3]:
phiA=(pA-1)*(qA-1);phiA

1960

In [4]:
phiB=(pB-1)*(qB-1);phiB

1008

Khóa bí mật dA và dB của Alice & Bob tương ứng là:

In [5]:
dA=inverse_mod(eA,phiA);dA

647

In [6]:
dB=inverse_mod(eB,phiB);dB

835

Alice mã hóa thông điệp m=331: m^eB (mod nB)

In [7]:
m=331

In [8]:
cA=power_mod(m,eB,nB);cA

829

Alice đồng thời tạo chữ ký: sA=m^dA (mod nA)

In [9]:
sA=power_mod(m,dA,nA);sA 

1235

In [0]:
Bob nhận được (cA,sA). Bob thực hiện:
Bước 1: Giải mã, tính cA^dB (mod nB)  được giá trị m1
Bước 2: Xác thực chữ ký, tính sA^eA (mod nA) được giá trị m2
        Nếu m1=m2 thì đây đúng là Alice !!! và bản rõ mà Alice muốn gửi là m1.
        Nếu m1 khác m2: Đã có giả mạo.

In [10]:
m1=power_mod(cA,dB,nB);m1

331

In [11]:
m2=power_mod(sA,eA,nA);m2

331

In [12]:
m1==m2

True

Tính toán thực hành RSA, RSA-DSA một cách hệ thống:


Sinh khóa:
Input: s (cỡ của số nguyên tố p,q <s)
Output: [n,phi,[e,n],[d,n]], với n=pq, phi=(p-1)(q-1),
        [e,n] là khóa công khai, [d,n] khóa bí mật.

In [13]:
def keygen(s):
    p=random_prime(s);q=random_prime(s);
    if p!=q:
        n=p*q;
        phi=(p-1)*(q-1);
        ch=true;
        while ch:
            e=randint(2,phi);
            if gcd(e,phi)==1:
                ch=false
        d=xgcd(e,phi)[1] % phi;
    return([n,phi,[e,n],[d,n]])

In [15]:
keygen(10000)

[4352759, 4345704, [3488935, 4352759], [2227543, 4352759]]

Khóa cho Alice

In [16]:
a=keygen(10000);a

[1668859, 1662640, [554219, 1668859], [782419, 1668859]]

In [17]:
nA=a[0];nA

1668859

In [18]:
phiA=a[1];phiA

1662640

In [19]:
eA=a[2][0];eA

554219

In [20]:
dA=a[3][0];dA

782419

Văn bản cần ký, m.

In [21]:
m=randint(2,2^10);m

122

Alice công khai n, e, bí mật d.
Alice ký m bằng chữ ký d, tính m^d rồi gửi cho Bob. 
Bob có (m^d,e) từ Alice.

Khi đó Bob có thể tính m thông qua đẳng thức m=(m^d)^e.

Vấn đề: Nếu kẻ thứ 3, Eve, thay đổi giá trị của m^d bằng XYZ. Khi đó Bob tính (XYZ)^e sẽ ra một giá trị khác m.

Từ đó chúng cần có cách để Bob có thể biết được giá trị m^d nhận được từ Alice là giá trị thật sự. Cần "chữ ký số".

In [22]:
mA=power_mod(m,dA,nA);mA

1651540

Bob muốn đọc văn bản gốc của Alice

In [23]:
m1=power_mod(mA,eA,nA);m1

122

In [24]:
m1==m

True

Vì sao cần chữ ký?

Giả sử có kẻ thứ 3, Eve, có thể thay đổi giá trị mA, khi đó giá trị mB=R1(mA)^d1 mà Bob tính được sẽ không bằng văn bản gốc.

Để tránh điều này, khi gửi Alice mA, cô ấy cần gửi thêm nA=m^e2 (mod n2)

Trong đó, Bob có khóa công khai (e2,n2) và khóa bí mật (d2,n2):

Như vậy Alice gửi cho Bob (mA,nA), nA được coi như là chữ ký.

Bob có thể xác thực chữ ký bằng cách kiểm tra:

mB == (nA)^d2
(vế phải = (m^e2)^d2 = m, mB=(mA)^e1). 

In [25]:
bob=keygen(20000);bob

[89855663, 89832000, [47075347, 89855663], [12710683, 89855663]]

In [26]:
nB=bob[0];nB

89855663

In [27]:
phiB=bob[1];phiB

89832000

In [28]:
eB=bob[2][0];eB

47075347

In [29]:
dB=bob[3][0];dB

12710683

Alice gửi cho Bob

In [30]:
m

122

In [31]:
def Alice_sign(m):
    return [power_mod(m,eB,nB),power_mod(m,dA,nA)]

In [32]:
A=Alice_sign(m);A

[51375991, 1651540]

Bob nhận được A=(mA,sA), kiểm chứng nểu 
mA^eA (mod nA) = sA^dB (mod nB)
True thì cho m, False thì thông báo "Có vấn đề".

In [37]:
def Bob_check(A):
    # Lỗi sai ở đây, ta phải đổi chỗ A[0] và A[1] cho nhau!
    if power_mod(A[1],eA,nA)==power_mod(A[0],dB,nB):
        print('The signature is right and the message is:',power_mod(A[0],eA,nA) )
    else:
        print('Something wong!')

In [38]:
Bob_check(A)

The signature is right and the message is: 567646


Thêm một ví dụ khác:

In [39]:
m=randint(2,min(n1,n2));m

NameError: name 'n1' is not defined

In [44]:
A=Alice_sign(m);A

[1461556, 16355958]

In [45]:
Bob_check(A)

The signature is right and the message is: 1714096


Thử một ví dụ khác khủng hơn xem?

In [46]:
a=keygen(10^100);a

[3827034825948845169923395450385007254256765516007836189124335026966898680416946208125470166126464874874830277903069789139951361274181817706282781899928095384623010920147284528435978223280095524290961,
 3827034825948845169923395450385007254256765516007836189124335026966898680416946208125470166126464868516204686515751446664259133515149218407114790278182095660364337876638016673802698063383344396863924,
 [513294563058303109492153561209108237342645098007775807710197252294257963677028446685260863660490668759274575053703779584833694267968733011133059813143212096006598879710131847555676158999332379578327,
  3827034825948845169923395450385007254256765516007836189124335026966898680416946208125470166126464874874830277903069789139951361274181817706282781899928095384623010920147284528435978223280095524290961],
 [62029233107258399065580593153441348393864835173378560063218353334790539087724521457742860691713286899395481925110429742008732531222857364517591508989004220361277911540359169674258083028932

In [47]:
n1=a[0];n1

3827034825948845169923395450385007254256765516007836189124335026966898680416946208125470166126464874874830277903069789139951361274181817706282781899928095384623010920147284528435978223280095524290961

In [48]:
phi1=a[1];phi1

3827034825948845169923395450385007254256765516007836189124335026966898680416946208125470166126464868516204686515751446664259133515149218407114790278182095660364337876638016673802698063383344396863924

In [49]:
e1=a[2][0];e1

513294563058303109492153561209108237342645098007775807710197252294257963677028446685260863660490668759274575053703779584833694267968733011133059813143212096006598879710131847555676158999332379578327

In [50]:
d1=a[3][0];d1

620292331072583990655805931534413483938648351733785600632183533347905390877245214577428606917132868993954819251104297420087325312228573645175915089890042203612779115403591696742580830289325393916819

In [51]:
bob=keygen(10^100);bob

[26170523785936606482105219854389757191242917626323645638409381093259435419761108601226980841078232092157847021518189115487245317861527204699675850852214632284204277598897736472008246662978989800413017,
 26170523785936606482105219854389757191242917626323645638409381093259435419761108601226980841078232081911079403520417581528930726170128436954068532307837627798585610808628895123300067387990796353530320,
 [24717232245166293786057711575303213818278498725409144233825204338904309286519866362536713596169246196164696762663794486963092302178411388946637663763566871800554809909326081584395679720781164028629677,
  26170523785936606482105219854389757191242917626323645638409381093259435419761108601226980841078232092157847021518189115487245317861527204699675850852214632284204277598897736472008246662978989800413017],
 [725940275495473388713509565403414839364685087107054742780410398661806276803096823777302942999135717114697558194637152808650114423654747901128032805066110338001897682467800296096080545

In [52]:
n2=bob[0];n2

26170523785936606482105219854389757191242917626323645638409381093259435419761108601226980841078232092157847021518189115487245317861527204699675850852214632284204277598897736472008246662978989800413017

In [53]:
phi2=bob[1];phi2

26170523785936606482105219854389757191242917626323645638409381093259435419761108601226980841078232081911079403520417581528930726170128436954068532307837627798585610808628895123300067387990796353530320

In [54]:
e2=bob[2][0];e2

24717232245166293786057711575303213818278498725409144233825204338904309286519866362536713596169246196164696762663794486963092302178411388946637663763566871800554809909326081584395679720781164028629677

In [55]:
d2=bob[3][0];d2

7259402754954733887135095654034148393646850871070547427804103986618062768030968237773029429991357171146975581946371528086501144236547479011280328050661103380018976824678002960960805450787179951566613

In [56]:
R1 = IntegerModRing(n1);R1

Ring of integers modulo 3827034825948845169923395450385007254256765516007836189124335026966898680416946208125470166126464874874830277903069789139951361274181817706282781899928095384623010920147284528435978223280095524290961

In [57]:
R2 = IntegerModRing(n2);R2

Ring of integers modulo 26170523785936606482105219854389757191242917626323645638409381093259435419761108601226980841078232092157847021518189115487245317861527204699675850852214632284204277598897736472008246662978989800413017

In [58]:
m=randint(2,min(n1,n2));m

1618290619784131321815400719623791911450323960175753931451503643205821555782516591993944020831597399994311490652506428308168222405977264376900284690345914446621532198180588587615000089461145305178469

In [59]:
A=Alice_sign(m);A

[6089405082367757565147764892026259210150575598342588221885394616348529355488557551806270910842038570672237449035442897730094200928255848250016325539623297545281860034883151353106723589123213318941,
 14591640043130240529229157974925713957587300395071281031811439604002062027803980399302393080807432335765850548143740117598148544415654178922091520573506643041263934200423019075749261893582421616831812]

In [60]:
Bob_check(A)

The signature is right and the message is: 1618290619784131321815400719623791911450323960175753931451503643205821555782516591993944020831597399994311490652506428308168222405977264376900284690345914446621532198180588587615000089461145305178469


In [1]:
factor(423642238748328749873248732)

2^2 * 43 * 2463036271792609010890981

In [2]:
factor(843759874385784375985948543684586768758)

2 * 26610173339 * 15854084519418852929290731361

In [3]:
factor(1522605027922533360535618378132637429718068114961380688657908494580122963258952897654000350692006130)

2 * 5 * 1621 * 613421 * 1687400747 * 123757232614736397551848335865061147 * 733257947272742522541754049165068869512409677